In [63]:
import cv2
import os
import numpy as np
import pickle
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tqdm.notebook import tqdm

Let us first read the train and test files

In [52]:
train_images_filenames = pickle.load(open('train_images_filenames.dat','rb'))
test_images_filenames = pickle.load(open('test_images_filenames.dat','rb'))
train_images_filenames = ['..' + n[15:] for n in train_images_filenames]
test_images_filenames  = ['..' + n[15:] for n in test_images_filenames]
train_labels = pickle.load(open('train_labels.dat','rb')) 
test_labels = pickle.load(open('test_labels.dat','rb'))

n_train = len(train_images_filenames)
n_test = len(test_images_filenames)
tradeoff = n_train/(n_train+n_test)
print(f"There are {tradeoff*100:.2f}% of training images.")

There are 69.97767857142857% of training images.


We create a KAZE/SIFT/.... object detector and descriptor

In [88]:
Detector = cv2.SIFT_create(nfeatures=100)
#Detector = cv2.KAZE_create(threshold=0.0001)

In [89]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

We compute the  descriptors for all the train images and subsequently build a numpy array with all the descriptors stacked together

In [55]:
for file in range(len(train_images_filenames)):
    train_images_filenames[file] = train_images_filenames[file][3:]

for file in range(len(test_images_filenames)):
    test_images_filenames[file] = test_images_filenames[file][3:]

Normal SIFT/KAZE

In [93]:
Train_descriptors = []
Train_label_per_descriptor = []

for filename,labels in zip(train_images_filenames,train_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    kpt,des=Detector.detectAndCompute(gray,None)
    Train_descriptors.append(des)
    Train_label_per_descriptor.append(labels)


In [94]:
Test_descriptors = []
Test_label_per_descriptor = []

for filename,labels in zip(test_images_filenames,test_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    kpt,des=Detector.detectAndCompute(gray,None)
    Test_descriptors.append(des)
    Test_label_per_descriptor.append(labels)

In [58]:
with open("pkls/test_des_sift_B", "wb") as file:
    pickle.dump(Test_descriptors, file)
with open("pkls/train_des_sift_B", "wb") as file:
    pickle.dump(Train_descriptors, file)

Compare the matches of each test image to training images and set the label with most matches.

In [95]:
test_results = []
i = 0
for test_label, test_des in zip(Test_label_per_descriptor, Test_descriptors):
    i += 1
    print(f"Matching Process: {i/len(Test_label_per_descriptor) * 100:.2f}%")
    best_num_match = 0
    best_match_label = None
    for train_label, train_des in zip(Train_label_per_descriptor, Train_descriptors):
        total_matches = 0
        matches = bf.knnMatch(test_des, train_des, k=2)
        
        # Apply ratio test
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
        total_matches = len(good_matches)
        if total_matches > best_num_match:
            best_num_match = total_matches
            best_match_label = train_label

    test_results.append(best_match_label)

with open("pkls/test_results_sift_B.pkl", "wb") as file:
    pickle.dump(test_results, file)

Matching Process: 0.12%
Matching Process: 0.25%
Matching Process: 0.37%
Matching Process: 0.50%
Matching Process: 0.62%
Matching Process: 0.74%
Matching Process: 0.87%
Matching Process: 0.99%
Matching Process: 1.12%
Matching Process: 1.24%
Matching Process: 1.36%
Matching Process: 1.49%
Matching Process: 1.61%
Matching Process: 1.73%
Matching Process: 1.86%
Matching Process: 1.98%
Matching Process: 2.11%
Matching Process: 2.23%
Matching Process: 2.35%
Matching Process: 2.48%
Matching Process: 2.60%
Matching Process: 2.73%
Matching Process: 2.85%
Matching Process: 2.97%
Matching Process: 3.10%
Matching Process: 3.22%
Matching Process: 3.35%
Matching Process: 3.47%
Matching Process: 3.59%
Matching Process: 3.72%
Matching Process: 3.84%
Matching Process: 3.97%
Matching Process: 4.09%
Matching Process: 4.21%
Matching Process: 4.34%
Matching Process: 4.46%
Matching Process: 4.58%
Matching Process: 4.71%
Matching Process: 4.83%
Matching Process: 4.96%
Matching Process: 5.08%
Matching Process

Save the resutls as pkl.

In [96]:
print(test_results)

['coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'Opencountry', 'coast', 'coast', 'coast', 'Opencountry', 'coast', 'coast', 'coast', 'coast', 'Opencountry', 'coast', 'Opencountry', 'coast', 'Opencountry', 'coast', 'coast', 'Opencountry', 'Opencountry', 'coast', 'coast', 'coast', 'coast', 'coast', 'Opencountry', 'coast', 'Opencountry', 'coast', 'tallbuilding', 'mountain', 'highway', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'inside_city', 'highway', 'coast', 'mountain', 'coast', 'Opencountry', 'coast', 'coast', 'Opencountry', 'tallbuilding', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'coast', 'mountain', 'coast', 'coast', 'coast', 'inside_city', 'coast', 'Opencountry', 'coast', 'Opencountry', 'coast', 'street', 'highway', 'coast', 'coast', 'inside_city', 'coast', 'coast', 'coast', 'coast', 'coast', 'Opencountry', 'Opencountry', 'coast', 'coast', 'coast

Check Accuracy

In [97]:
num_corrects = 0
for test_label, test_result in zip(Test_label_per_descriptor, test_results):
    num_corrects += 1 if test_label == test_result else 0

acc = num_corrects/len(Test_label_per_descriptor)
print(f"Accuracy: {acc}")

Accuracy: 0.298636926889715


F1-Score

In [98]:
classes = list(set(Test_label_per_descriptor))

f1_i = 0
for classe in classes:
    TP = 0
    FP = 0
    FN = 0
    for test_label, result_label in zip(Test_label_per_descriptor, test_results):
        TP += 1 if test_label == classe and result_label == classe else 0
        FP += 1 if test_label != classe and result_label == classe else 0
        FN += 1 if test_label == classe and result_label != classe else 0

    P_i = TP / (TP + FP)
    R_i = TP / (TP + FN)
    f1_i += 2 * (P_i * R_i) / (P_i + R_i)  
    
f1 = f1_i / len(classes)
print(f"F1 score = {f1}")
            


F1 score = 0.288120258859093
